# Problema da Mochila Múltiplas com o Gurobi

Temos um conjunto de itens, em que cada item tem um peso e um valor. Queremos escolher com variáveis binárias, quais desses itens serão colocados em cada mochila de forma que o peso total de cada mochila não exceda a capacidade da mochila, que seja restrigindo a colocação dos itens em uma única mochila, e que seja maximizado o valor total dos itens em cada mochila.


Maximizar 

$\sum_{i=1}^{n}\sum_{j=1}^{m}v_i x_{ij}$

Sujeito a: 

$\sum_{i=1}^{n}\sum_{j=1}^{m}w_i x_{ij} \leq C_j, $ $\forall i=1, ..., n $

$\sum_{j=1}^{m}x_{ij} \leq 1, $ $\forall j=1, ..., m $

$x_{ij} \in \{0, 1\}$ $\forall i=1, ..., m $; $j=1, ..., n $


com $w_i$ e $v_i$ corresponde ao peso e valor do item i, respectivamente.

In [6]:
import gurobipy as gp

In [15]:
name_arq = 'dados_problema_da_mochila_multipla.txt'
with open(name_arq, 'r') as f:
    lines = f.readlines()        
    # Lista com a capacidade de cada mochila    
    capacidade = {f'Mochila_{idx}':int(line) for idx, line in enumerate(lines[0].strip().split(' '), 1)}
    mochilas = [cap for cap in capacidade]
    # Excluindo o primeiro elemento da lista 'lines'
    del(lines[0])
    # Lista de itens
    itens = [f'Item_{i}' for i in range(1,len(lines)+1)]        
    # Construindo dicionário pesos e valores
    pesos, valores = dict(), dict()
    for i, line in enumerate(lines):
        peso, valor = line.strip().split(' ')
        pesos[itens[i]] = int(peso)
        valores[itens[i]] = int(valor)        

In [26]:
# Criando o modelo
m = gp.Model()

# Configuração para não escrever o relatório
m.setParam(gp.GRB.Param.OutputFlag, 0)

# Inserindo as variáveis de decisão
x = m.addVars(itens, capacidade_mochilas, vtype=gp.GRB.BINARY)

# Função objetivo
m.setObjective(
    gp.quicksum(x[i,j]*valores[i] for i in itens for j in mochilas),
    sense=gp.GRB.MAXIMIZE
)

# Restrição cada item em uma única mochila
c1 = m.addConstrs(
    gp.quicksum(x[i,j] for j in mochilas) <= 1 for i in itens
)

# Restrição da capacidade
c2 = m.addConstrs(
    gp.quicksum(x[i,j]*pesos[i] for i in itens) <= capacidade[j] for j in mochilas
)

# Executa o modelo
m.optimize()

In [33]:
# Valor objetivo
print('Valor Total', m.objVal, '\n')

# Mostra os itens que estão em cada mochila e capacidade utilizada
for j in mochilas:
    print(f'Itens na {j}:')
    for i in itens:
        if round(x[i,j].X) == 1:
            print(i, ' ', end='')
    print('\nCapacidade utilizada:',capacidade[j] - c2[j].Slack)
    print('\n')

Valor Total 13364.0 

Itens na Mochila_1:
Item_4  Item_5  Item_9  Item_25  Item_44  Item_47  Item_48  Item_49  Item_53  Item_59  Item_62  Item_63  
Capacidade utilizada: 1499.0


Itens na Mochila_2:
Item_2  Item_7  Item_11  Item_12  Item_13  Item_14  Item_16  Item_19  Item_22  Item_23  Item_28  Item_29  Item_31  Item_32  Item_38  Item_39  Item_50  Item_60  
Capacidade utilizada: 2000.0


Itens na Mochila_3:
Item_10  Item_15  Item_18  Item_24  Item_27  Item_30  Item_35  Item_40  Item_41  Item_46  Item_51  Item_57  
Capacidade utilizada: 1700.0


